#Unblanced data 


i had to balance the data so here you could find new data and colab notebook  


 colab : [colab.research.google.com/drive/141XZ18NzQHcgkV1BAceFY43m9q6lveSv?usp=sharing](http://)
 
 newdata : https://drive.google.com/drive/folders/1lV_LIy2XPnIHC5Kcd_d4PsE8sNs434U6?usp=sharing

In [1]:
#import libraries 
import tensorflow as tf 
import tensorflow_datasets as tfds
import keras 
import numpy as np 
import matplotlib.pyplot as plt


import pandas as pd 
from keras.applications import VGG19,Xception,VGG16
from keras.layers import Dense , Conv2D , MaxPooling2D , Dropout,Flatten,Convolution2D
from keras.models  import Sequential
from keras.preprocessing.image import ImageDataGenerator



In [3]:
#access my drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
#call vgg model
vgg_model =  VGG19(include_top=True , weights='imagenet')
for models in vgg_model.layers:
  models.trainable= False

303128576/574710816 [==============>...............] - ETA: 2:04

KeyboardInterrupt: 

In [ ]:
#converting from functionally model to sequential model
#removing the last 2 alyer to get rid of output layer in VGG16
vgg_model = keras.Model(inputs=vgg_model.input, outputs=vgg_model.layers[-2].output)
model = keras.Sequential()
for layer in vgg_model.layers:
  model.add(layer)

In [ ]:
#add trianbles layers
model.add(Dense(4056, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer="adam", loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)


In [ ]:
train_data = defect_tree = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/My Drive/archive_(1)/images',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training",
    interpolation="bilinear",
    follow_links=False,
)
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/My Drive/archive_(1)/images',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
)

In [ ]:
batch_size =32
model.fit(train_data,
    validation_data = test_data, 
    callbacks=[early],
    epochs = 50)

In [ ]:
#evulate mmodel
model.evaluate(test_data)

In [ ]:
#predict model
y_pred = np.array([])
y_true = np.array([])
i = 0

for image,label in test_data : 
  i+=1
  y = model.predict(image)
  y = np.argmax(y,axis=1)
  y_true = np.append(y_true,label)
  y_pred = np.append(y_pred,y)
  if i == 176 // 32 + 1:
    break


In [ ]:
from  sklearn .metrics import classification_report,confusion_matrix
report=classification_report(y_true,y_pred)
print(report)

In [ ]:
cm= confusion_matrix(y_true,y_pred)
print(cm)

In [ ]:
import pandas as pd
import seaborn 
df_cm = pd.DataFrame(cm, index = [i for i in [0,1]],
                  columns = [i for i in [0,1]])
seaborn .heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='d')
plt.title('confusion matrix')
plt.xlabel('prediction')
plt.ylabel('Actual');

In [ ]:
model.save("textile.h5")